In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder



In [66]:
data_c = pd.read_csv("output_c.txt", header=None)
data_h = pd.read_csv("output_h.txt", header=None)
data_2 = pd.read_csv("output_2.txt", header=None)

data = pd.concat([data_c, data_h, data_2], ignore_index=True)

data.to_csv("concatenated_data.txt", index=False, header=False)


features = data.iloc[:, :-1].values
data.iloc[:, -1] = data.iloc[:, -1].astype(str)
labels = data.iloc[:, -1].values


le = LabelEncoder()
labels = le.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [47]:

class GestureCNN(nn.Module):
    def __init__(self):
        super(GestureCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(16 * 6 * 6, 64)
        self.fc2 = nn.Linear(64, 3)  # 3 output classes: c, h, 2

    def forward(self, x):
        x = F.relu(self.conv1(x))   # [B, 8, 6, 6]
        x = F.relu(self.conv2(x))   # [B, 16, 6, 6]
        x = x.view(x.size(0), -1)   # flatten
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [48]:

class GestureDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).reshape(-1, 1, 6, 6)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [49]:
from torch.utils.data import DataLoader

train_dataset = GestureDataset(X_train, y_train)
test_dataset = GestureDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GestureCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# Training loop
for epoch in range(20):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()
        out = model(batch_x)
        loss = loss_fn(out, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 42.5348
Epoch 2, Loss: 8.5037
Epoch 3, Loss: 4.0152
Epoch 4, Loss: 1.0746
Epoch 5, Loss: 0.3632
Epoch 6, Loss: 0.4222
Epoch 7, Loss: 0.1190
Epoch 8, Loss: 0.1612
Epoch 9, Loss: 0.0862
Epoch 10, Loss: 0.0584
Epoch 11, Loss: 0.0529
Epoch 12, Loss: 0.0351
Epoch 13, Loss: 0.0301
Epoch 14, Loss: 0.0243
Epoch 15, Loss: 0.0193
Epoch 16, Loss: 0.0126
Epoch 17, Loss: 0.0086
Epoch 18, Loss: 0.0096
Epoch 19, Loss: 0.0047
Epoch 20, Loss: 0.0030


In [51]:
model.eval()
correct = total = 0
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        pred = model(batch_x).argmax(dim=1)
        correct += (pred == batch_y).sum().item()
        total += batch_y.size(0)

print(f"Test Accuracy: {correct / total:.2%}")


Test Accuracy: 100.00%


In [52]:
def predict(image_array):
    image_tensor = torch.tensor(image_array, dtype=torch.float32).reshape(1, 1, 6, 6).to(device)
    model.eval()
    with torch.no_grad():
        out = model(image_tensor)
        pred_class = out.argmax(dim=1).item()
    return le.inverse_transform([pred_class])[0]


In [53]:
input_vector = np.array([254, 0, 254, 76, 254, 254,
                         252, 110, 254, 0, 254, 254,
                         254, 37, 54, 1, 254, 154,
                         10, 0, 0, 111, 166, 254,
                         83, 0, 254, 0, 0, 254,
                         147, 172, 4, 0, 254, 251])


In [54]:
predict(input_vector)

'2'

In [64]:
input_2 = np.array([838, 842, 842, 710, 516, 365,
 831, 842, 842, 805, 314, 333,
 559, 567, 640, 631, 617, 357,
 380, 312, 343, 386, 382, 387,
 412, 477, 402, 602, 589, 326,
 822, 842, 842, 641, 558, 340]
)

predict(input_2)

'c'